# IBM Data Science Course
### Final Module - Applied Data Science Capstone
Lucas Friedrich Meincke<br><br>

This Notebook will be used for the Capstone Project which is the last module of the IBM Data Science Course on Coursera.

In [48]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import requests
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
#pd.set_option('display.max_rows', 1000)
#pd.set_option('display.max_columns', 1000)

In [49]:
link1 = requests.get('https://en.wikipedia.org/wiki/List_of_cities_in_Germany_by_population').text
soup1 = BeautifulSoup(link1, 'html.parser')

l = []

for td in soup1.find('tbody').find_all('td'):
    data = td.text.strip()
    data = data.replace(',', '')
    data = data.split(" / ", 1)
    data = data[-1]
    data = data.split(" / ", 1)
    data = data[0]
    data = data.replace('°N', ';')
    data = data.replace('°E', '')
    data = data.replace('\ufeff', '')
    data = data.replace('\xa0', '')
    l.append(data)

rows = int((len(l)/9)) # number of rows is the lengh of the list / number of columns (converted to int)
l = np.reshape(l, (rows, 9)) # reshaping the list into matrix

df_cities = pd.DataFrame(l, columns = ["Rank", "City", "State", "Population", "2011 Census", 
                                      "Change", "Land Area", "Pop. Dens.", "Location"])

df_cities[['Latitude','Longitude']] = df_cities.Location.str.split(";", expand=True,).astype('float').round(3)
df_cities[["City", "State"]] = df_cities[["City", "State"]].astype('string')
df_cities["Population"] = df_cities["Population"].astype('int')

df_cities = df_cities[["City", "State", "Population", "Latitude", "Longitude"]]
df_cities.replace({'Halle (Saale)': 'Saale', 'Munich (München)': 'München', 'Cologne (Köln)': 'Köln',
                  'Hanover (Hannover)': 'Hannover', 'Nuremberg (Nürnberg)': 'Nürnberg'}, inplace=True)
df_cities

,City,State,Population,Latitude,Longitude
0,Berlin,Berlin,3520031,52.517,13.383
1,Hamburg,Hamburg,1787408,53.550,10.000
2,München,Bavaria,1450381,48.133,11.567
3,Köln,North Rhine-Westphalia,1060582,50.933,6.950
4,Frankfurt am Main,Hesse,732688,50.117,8.683
5,Stuttgart,Baden-Württemberg,623738,48.783,9.183
6,Düsseldorf,North Rhine-Westphalia,612178,51.233,6.783
7,Dortmund,North Rhine-Westphalia,586181,51.517,7.467
8,Essen,North Rhine-Westphalia,582624,51.450,7.017
9,Leipzig,Saxony,560472,51.333,12.383


In [3]:
list_cities = list(df_cities["City"])
l = []
count = 0

for x in list_cities:
    try:
        link2 = requests.get(f'https://de.wikipedia.org/wiki/{x}').text
        soup2 = BeautifulSoup(link2, 'html.parser')
        tables = soup2.find('table', style='border: 5px solid #E5E5E5; font-size:95%; background:#E5E5E5; margin-bottom:10px;')  
        for row in tables.find_all('tr'):
            for td in row.find_all('td'):
                data = td.text.strip()
                data = data.replace('Jan', '').replace('Feb', '').replace('Mär', '').replace('Apr', '')
                data = data.replace('Mai', '').replace('Jun', '').replace('Jul', '').replace('Aug', '')
                data = data.replace('Sep', '').replace('Okt', '').replace('Nov', '').replace('Dez', '')
                data = data.replace(',', '.')
                data = data.replace('−', '-')
                l.append(data)
                l = list(filter(None, l))
            if data=="":
                continue
            else: l.append(x)
        print(x)
    except:
        count = count+1
        print('error')
        
print(count)
rows = int((len(l)/16)) # number of rows is the lengh of the list / number of columns (converted to int)
l = np.reshape(l, (rows, 16)) # reshaping the list into matrix

Berlin
Hamburg
München
Köln
Frankfurt am Main
Stuttgart
Düsseldorf
Dortmund
Essen
Leipzig
Bremen
Dresden
Hannover
Nürnberg
Duisburg
Bochum
Wuppertal
Bielefeld
Bonn
Münster
Karlsruhe
Mannheim
Augsburg
Wiesbaden
Gelsenkirchen
Mönchengladbach
Braunschweig
Chemnitz
Kiel
Aachen
error
Magdeburg
Freiburg im Breisgau
Krefeld
Lübeck
Oberhausen
Erfurt
Mainz
Rostock
Kassel
error
Hamm
Saarbrücken
error
Potsdam
error
error
Leverkusen
Osnabrück
Solingen
Heidelberg
Herne
Neuss
Darmstadt
error
Regensburg
error
Würzburg
error
error
Offenbach am Main
Ulm
Heilbronn
Pforzheim
error
error
Trier
error
error
Bremerhaven
Koblenz
Bergisch Gladbach
error
error
error
error
Siegen
error
error
Kaiserslautern
19


In [4]:
df_clima = pd.DataFrame(l, columns=["Measure", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul",
                        "Aug", "Sep", "Oct", "Nov", "Dec", "Sum/Avg", "Total", "City"])

df_clima = df_clima.astype({'Measure': 'string', 'Jan': 'float64', 'Feb': 'float64', 'Mar': 'float64',
                           'Apr': 'float64', 'May': 'float64', 'Jun': 'float64', 'Jul': 'float64',
                           'Aug': 'float64', 'Sep': 'float64', 'Oct': 'float64', 'Nov': 'float64',
                           'Dec': 'float64', 'Sum/Avg': 'string', 'Total': 'float64', 'City': 'string'})

In [5]:
mask = df_clima['Measure'].isin(['Regentage (d)', 'Sonnenstunden (h/d)', 'Luftfeuchtigkeit (%)',
                          'Temperatur (°C)', 'Rekordmaximum (°C)', 'Rekordminimum (°C)'])
df_clima = df_clima[~mask].reset_index(drop=True)
df_clima["Measure"] = df_clima['Measure'].replace(["Max. Temperatur (°C)", "Min. Temperatur (°C)", "Niederschlag (mm)"],
                                      ["Max. Temp. (°C)", "Min. Temp. (°C)", "Precipitation (mm)"])
df_clima

,Measure,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Sum/Avg,Total,City
0,Max. Temp. (°C),2.9,4.2,8.5,13.2,18.9,21.6,23.7,23.6,18.8,13.4,7.1,4.4,Ø,13.4,Berlin
1,Min. Temp. (°C),-1.9,-1.5,1.3,4.2,9.0,12.3,14.3,14.1,10.6,6.4,2.2,-0.4,Ø,5.9,Berlin
2,Precipitation (mm),42.3,33.3,40.5,37.1,53.8,68.7,55.5,58.2,45.1,37.3,43.6,55.3,Σ,570.7,Berlin
3,Max. Temp. (°C),3.5,4.4,8.0,12.3,17.5,19.9,22.1,22.2,17.9,13.0,7.5,4.6,Ø,12.8,Hamburg
4,Min. Temp. (°C),-1.4,-1.2,1.1,3.3,7.4,10.5,12.7,12.5,9.6,6.0,2.4,0.0,Ø,5.3,Hamburg
5,Precipitation (mm),64.4,42.4,62.9,45.6,53.7,76.9,74.7,73.0,68.4,63.6,69.4,77.7,Σ,772.7,Hamburg
6,Max. Temp. (°C),4.7,7.5,10.8,16.2,19.5,24.4,26.2,26.2,20.5,15.3,9.2,6.8,Ø,15.6,München
7,Min. Temp. (°C),-1.6,0.1,2.2,5.5,9.3,13.8,15.2,14.9,10.5,6.6,2.7,0.4,Ø,6.7,München
8,Precipitation (mm),71.0,67.0,43.0,52.0,137.0,101.0,103.0,113.0,83.0,65.0,53.0,45.0,Σ,933.0,München
9,Max. Temp. (°C),5.2,6.6,10.5,14.2,19.0,21.3,23.7,23.7,19.6,14.6,9.0,6.2,Ø,14.5,Köln


In [6]:
df_clima.groupby('Measure')['City'].nunique()

Measure
Max. Temp. (°C)       59
Min. Temp. (°C)       59
Precipitation (mm)    60
Name: City, dtype: int64

In [50]:
df_main = df_clima[['City', 'Measure']]
df_main['Total'] = (df_clima['Mar'] + df_clima['Apr'] + df_clima['May'] + df_clima['Jun'] + 
                         df_clima['Jul'] + df_clima['Aug'] + df_clima['Sep'] + df_clima['Oct'])

df_main = df_main.pivot(index='City', columns='Measure')['Total'].reset_index().round(3)
df_main["Summed Avg. Temp. (°C)"] = ((df_main["Max. Temp. (°C)"] + df_main["Min. Temp. (°C)"])/2/8).round(2)
df_main = pd.merge(df_main, df_cities, on ='City', how ='inner')
df_main = df_main[["City", "Precipitation (mm)", "Summed Avg. Temp. (°C)", "Population", "Latitude", "Longitude"]].round(2).sort_values(
                                by=["Precipitation (mm)", "Summed Avg. Temp. (°C)"], ascending=[True, False])
df_main = df_main.dropna().reset_index(drop=True)
df_main = df_main.head(12)
df_main

<ipython-input-50-8a1e13a8aa87>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clima_summ['Total'] = (df_clima['Mar'] + df_clima['Apr'] + df_clima['May'] + df_clima['Jun'] +


,City,Precipitation (mm),Summed Avg. Temp. (°C),Population,Latitude,Longitude
0,Magdeburg,347.6,12.96,235723,52.13,11.62
1,Leipzig,357.0,13.93,560472,51.33,12.38
2,Nürnberg,371.0,14.13,509975,49.45,11.08
3,Mainz,377.0,13.61,209779,50.00,8.27
4,Pforzheim,377.6,13.91,122247,48.90,8.72
5,Erfurt,384.0,12.03,210118,50.98,11.03
6,Dresden,387.0,14.74,543825,51.03,13.73
7,Berlin,396.2,13.37,3520031,52.52,13.38
8,Heidelberg,401.0,15.48,156267,49.42,8.72
9,Mannheim,401.0,15.21,305780,49.48,8.47


In [8]:
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode('Germany')
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.0834196 10.4234469


In [58]:
# create map of Germany using latitude and longitude values
map_germany = folium.Map(location=[latitude, longitude], zoom_start=8)

# add markers to map
for lat, lng, city in zip(df_main['Latitude'], df_main['Longitude'], df_main['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_germany)  
    
map_germany

In [83]:
# credencials to be used on Foursquare API
CLIENT_ID = 'ZHZIJHIFEQCF4TUT3UOFXEVMSMTSOQ204ALBKXBCNMH2WXRA'
CLIENT_SECRET = '41I33OL4YE34QS5VWJXTNMSWCQFV143GKLFQ4OA1CQL0B2V4'
VERSION = '20180605'
LIMIT = 100 # A default Foursquare API limit value

In [80]:
# function to collect all the venues for each zip code
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City Name', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue ID',
                  'Venue Name', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# sending the parameters to the function
df_venues_all = getNearbyVenues(names=df_main['City'],
                            latitudes=df_main['Latitude'],
                            longitudes=df_main['Longitude'])

In [81]:
df_venues = df_venues_all[df_venues_all['Venue Category'].str.contains('Ice|Café|Coffee Shop|Bistro')].reset_index(drop=True)
#'Ice|Café|Coffee Shop|Bistro|Bar|Pub'
df_venues.replace({'Coffee Shop': 'Café', 'Bistro': 'Café'}, inplace=True)
df_venues

,City Name,City Latitude,City Longitude,Venue ID,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,Magdeburg,52.13,11.62,561a856e498ee6fc8b49cafb,Herzstück - Das Kuchenatelier,52.133728,11.615929,Café
1,Magdeburg,52.13,11.62,4cb5ddce1b0af04d8a64cc25,Il Capitello,52.126257,11.634307,Café
2,Magdeburg,52.13,11.62,4dc53e2345dd2645526a7f15,Eis-Konditorei Bortscheller,52.106756,11.640792,Ice Cream Shop
3,Leipzig,51.33,12.38,51e3cd828bbde0ebc2f0d32f,Handbrotzeit,51.341338,12.378071,Café
4,Leipzig,51.33,12.38,548137ca498eb80c2a45fc33,Espresso Zack Zack,51.332784,12.404315,Café
5,Leipzig,51.33,12.38,4c179756f256a59330c9ec3e,Eisdiele Pfeifer,51.322069,12.371843,Ice Cream Shop
6,Leipzig,51.33,12.38,4bf93c3c5efe2d7fdedb6b34,Milchbar Pinguin,51.341443,12.375083,Ice Cream Shop
7,Leipzig,51.33,12.38,4d5d74da9ac9a0937f987b94,Café Maître,51.323207,12.373430,Café
8,Leipzig,51.33,12.38,5b6d94b5c0cacb002c27f705,Dankbar Kaffee Leipzig,51.342186,12.361047,Café
9,Leipzig,51.33,12.38,54e434b7498e1961aef7c161,IMPERII,51.342215,12.379639,Café


In [120]:
def getVenuesRating(ID):
    
    rating_list=[]
    #for ids in ID:        

    url = 'https://api.foursquare.com/v2/venues/561a856e498ee6fc8b49cafb?&client_id={}&client_secret={}&v={}'.format(
        #ids,
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        LIMIT)
            
    results = requests.get(url).json()["response"]#["venue"]#["rating"]
        
    rating_list.append(results)
        #rating_list.append([(
         #   v['id'],
          #  v['rating"], 
           # v['ratingSignals']) for v in results])     
        
    return(results)

df_ratings = getVenuesRating(ID=df_venues['Venue ID'])

In [122]:
df_ratings

{}

In [115]:
rating_list.append([(
            v['venue']['id'],
            v['venue']['rating'], 
            v['venue']['ratingSignals']) for v in results])

    rating = pd.DataFrame([item for ratings_list in rating_list for item in ratings_list])
    rating.columns = ['Venue ID', 'Rating', 'Votes']
    
    #rating = pd.DataFrame(data=rating_list, dtype='float')
    #rating.columns = ['Rating']
    #print(results)

IndentationError: unexpected indent (<ipython-input-115-bc3068e34c79>, line 6)

In [116]:
# one hot encoding
venues_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_onehot['City Name'] = df_venues['City Name'] 

# move neighbourhood column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]

# creating the grouped and normalized dataframe to be used in the kmean function
venues_grouped = venues_onehot.groupby('City Name').mean().reset_index()
venues_grouped

,City Name,Café,Ice Cream Shop
0,Berlin,0.833333,0.166667
1,Bremen,0.692308,0.307692
2,Dresden,0.800000,0.200000
3,Erfurt,0.857143,0.142857
4,Heidelberg,0.727273,0.272727
5,Leipzig,0.857143,0.142857
6,Magdeburg,0.666667,0.333333
7,Mainz,0.615385,0.384615
8,Mannheim,0.937500,0.062500
9,Nürnberg,0.933333,0.066667


In [44]:
# set number of clusters. It has been chosen k = 5 as it returned the max number of different clusters
k = 5

venues_grouped_clustering = venues_grouped.drop('City Name', 1)

# run k-means clustering. It has been chosen n_init = 30 as it returned the max number of different clusters
kmeans = KMeans(n_clusters = k, n_init=30, random_state=0).fit(venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:2000]

array([4, 0, 1, 0, 4, 4, 0, 0, 0, 4, 4, 0, 0, 0, 1, 0, 1, 0, 0, 0, 3, 0,
       0, 0, 3, 2, 0, 4, 1, 1, 0, 0, 4, 4, 4, 4, 0, 4, 0, 1, 4, 0, 4, 3,
       4, 3, 0, 0, 4, 0, 1, 1, 0, 4, 4, 0, 1, 4])

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['City Name'] = venues_grouped['City Name']

for ind in np.arange(venues_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

# add clustering labels
venues_sorted.insert(0, "Cluster Labels", kmeans.labels_)

venues_merged = df_main

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
venues_merged = venues_merged.join(venues_sorted.set_index('City Name'), on='City')

#toronto_merged.head() # check the last columns!
venues_merged

,City,Precipitation (mm),Summed Avg. Temp. (°C),Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Magdeburg,347.6,12.96,235723,52.13,11.62,4,Drugstore,Supermarket,Hotel,Park,Bar,Italian Restaurant,Nightclub,Plaza,Mediterranean Restaurant,Greek Restaurant
1,Leipzig,357.0,13.93,560472,51.33,12.38,0,Café,Park,Hotel,Zoo Exhibit,Coffee Shop,Irish Pub,Falafel Restaurant,Chinese Restaurant,Vietnamese Restaurant,Plaza
2,Nürnberg,371.0,14.13,509975,49.45,11.08,0,Park,Coffee Shop,Café,Greek Restaurant,Gym / Fitness Center,Historic Site,German Restaurant,Burger Joint,Bakery,Hotel
3,Mainz,377.0,13.61,209779,50.00,8.27,4,Supermarket,Café,Ice Cream Shop,German Restaurant,Italian Restaurant,Wine Bar,Park,Plaza,Pizza Place,Bar
4,Pforzheim,377.6,13.91,122247,48.90,8.72,3,Supermarket,Drugstore,German Restaurant,Café,Fast Food Restaurant,Coffee Shop,Hotel,Train Station,Big Box Store,Furniture / Home Store
5,Erfurt,384.0,12.03,210118,50.98,11.03,1,Café,Supermarket,Drugstore,Gas Station,Plaza,Big Box Store,Park,Vegetarian / Vegan Restaurant,Italian Restaurant,German Restaurant
6,Dresden,387.0,14.74,543825,51.03,13.73,0,Plaza,German Restaurant,Coffee Shop,Park,Café,Hotel,Bakery,Scenic Lookout,Restaurant,Art Gallery
7,Berlin,396.2,13.37,3520031,52.52,13.38,0,Park,Coffee Shop,Bookstore,Hotel,Concert Hall,Bakery,Monument / Landmark,Cocktail Bar,Café,Pastry Shop
8,Heidelberg,401.0,15.48,156267,49.42,8.72,0,Hotel,German Restaurant,Café,Plaza,Ice Cream Shop,Italian Restaurant,Coffee Shop,Falafel Restaurant,Bar,Supermarket
9,Mannheim,401.0,15.21,305780,49.48,8.47,0,Café,Hotel,Park,Bar,Italian Restaurant,German Restaurant,Vietnamese Restaurant,Lounge,Plaza,Korean Restaurant


In [47]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(venues_merged['Latitude'], venues_merged['Longitude'], venues_merged['City'], venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
print(venues_grouped.columns)